In [6]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pickle

In [7]:
job_rec_1 = pd.read_excel('data/Job_Suggestions_1.xlsx')
job_rec_2 = pd.read_excel('data/Job_Suggestions_2.xlsx')
job_rec_3 = pd.read_excel('data/Job_Suggestions_3.xlsx')
job_rec_4 = pd.read_excel('data/Job_Suggestions_4.xlsx')

In [8]:
job_rec_1.head()

,Timestamp,Company,CompanyLinkedIn,City,Country,Job_Type,Job_Role,Job_Desc,Desc_LinkedIn,Salary
0,"May 13, 2015 at 11:20AM",TalentOne.in (Formerly HireAtEase),http://www.linkedin.com/company/201192,"Bengaluru Area, India",IN,Computer Software,Java + UI Developer/Lead/Architect,Summary The candidate will be involved in requ...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
1,"May 12, 2015 at 02:15PM",Crossover.,http://www.linkedin.com/company/9308035,"Bengaluru Area, India",IN,Human Resources,Global IT Sourcer - $30k,If you are an energetic and dynamic individual...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
2,"May 06, 2015 at 11:50AM",7C Studio,http://www.linkedin.com/company/3765462,"Bengaluru Area, India",IN,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,https://www.linkedin.com/jobs?viewJob=&jobId=6...,No salary provided
3,"May 15, 2015 at 11:00PM",Planetsurf Creations,http://www.linkedin.com/company/2532948,"Bengaluru Area, India",IN,"Internet, Marketing and Advertising",Junior Web Developer,"Candidate be involved in development, testing,...",https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
4,"May 06, 2015 at 11:50AM",7C Studio,http://www.linkedin.com/company/3765462,"Bengaluru Area, India",IN,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,https://www.linkedin.com/jobs?viewJob=&jobId=6...,No salary provided


In [9]:
main_suggestions_df = pd.concat([job_rec_1,job_rec_2,job_rec_3,job_rec_4],axis=0)
main_suggestions_df

,Timestamp,Company,CompanyLinkedIn,City,Country,Job_Type,Job_Role,Job_Desc,Desc_LinkedIn,Salary
0,"May 13, 2015 at 11:20AM",TalentOne.in (Formerly HireAtEase),http://www.linkedin.com/company/201192,"Bengaluru Area, India",IN,Computer Software,Java + UI Developer/Lead/Architect,Summary The candidate will be involved in requ...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
1,"May 12, 2015 at 02:15PM",Crossover.,http://www.linkedin.com/company/9308035,"Bengaluru Area, India",IN,Human Resources,Global IT Sourcer - $30k,If you are an energetic and dynamic individual...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
2,"May 06, 2015 at 11:50AM",7C Studio,http://www.linkedin.com/company/3765462,"Bengaluru Area, India",IN,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,https://www.linkedin.com/jobs?viewJob=&jobId=6...,No salary provided
3,"May 15, 2015 at 11:00PM",Planetsurf Creations,http://www.linkedin.com/company/2532948,"Bengaluru Area, India",IN,"Internet, Marketing and Advertising",Junior Web Developer,"Candidate be involved in development, testing,...",https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
4,"May 06, 2015 at 11:50AM",7C Studio,http://www.linkedin.com/company/3765462,"Bengaluru Area, India",IN,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,https://www.linkedin.com/jobs?viewJob=&jobId=6...,No salary provided
...,...,...,...,...,...,...,...,...,...,...
1888,"April 21, 2015 at 10:15AM",InfoCognitio,http://www.linkedin.com/company/9350037,"Bengaluru Area, India",IN,Information Technology and Services,Content Developer / Technical Writer,Be able to write/develop content on simple to ...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
1889,"April 17, 2015 at 11:40AM",TextNook,http://www.linkedin.com/company/3700902,"Bengaluru Area, India",IN,Information Technology and Services,Software Engineer,"As a platform builder, you will be responsible...",https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
1890,"April 07, 2015 at 08:30PM",Watchy technology pvt ltd.,http://www.linkedin.com/company/2260045,"Bengaluru Area, India",IN,Computer Hardware,Software Engineer - Network,Modify IP network stack on Linux Optimising TC...,https://www.linkedin.com/jobs?viewJob=&jobId=4...,No salary provided
1891,"April 25, 2015 at 04:45PM",HOWTODO GLOBAL CONSULTING SERVICES,http://www.linkedin.com/company/6497045,"Bengaluru Area, India",IN,Computer Software,SAP BASIS ENGINEER - IMMEDIATE JOINING,SAP BASIS ENGINEER for one of our CMM LEVEL 5 ...,https://www.linkedin.com/jobs?viewJob=&jobId=5...,No salary provided


In [10]:
def tokenize_stem(series):

    tokenizer =TreebankWordTokenizer()
    stemmer = PorterStemmer()
    series = series.apply(lambda x: x.replace("\n", ' '))
    series = series.apply(lambda x: tokenizer.tokenize(x))
    series = series.apply(lambda x: [stemmer.stem(w) for w in x])
    series = series.apply(lambda x: ' '.join(x))
    return series


In [ ]:

def display_topics(model, feature_names, no_top_words, topic_names=None):
    '''
    displays topics and returns list of toppics
    '''

    topic_list = []
    for i, topic in enumerate(model.components_):
        if not topic_names or not topic_names[i]:
            print("\nTopic ", i)
        else:
            print("\nTopic: '",topic_names[i],"'")

        print(", ".join([feature_names[k]
                        for k in topic.argsort()[:-no_top_words - 1:-1]]))
        topic_list.append(", ".join([feature_names[k]
                        for k in topic.argsort()[:-no_top_words - 1:-1]]))
    return model.components_, topic_list


In [ ]:

def return_topics(series, num_topics, no_top_words, model, vectorizer):
    '''
    returns document_topic matrix and topic modeling model
    '''
    #turn job into series
    series = tokenize_stem(series)
    #transform series into corpus
    ex_label = [e[:30]+"..." for e in series]
    #set vectorizer ngrams = (2,2)
    vec = vectorizer(stop_words = 'english')

    doc_word = vec.fit_transform(series)

    #build model
    def_model = model(num_topics)
    def_model = def_model.fit(doc_word)
    doc_topic = def_model.transform(doc_word)
    #print('model components: ', def_model.components_[0].shape)
    #print('doc_topic', doc_topic[0])
    model_components, topic_list = display_topics(def_model, vec.get_feature_names(), no_top_words)
    return def_model.components_, doc_topic, def_model, vec, topic_list#, topics

In [11]:
def process_data():
    '''
    uses the functions above to read in files, model, and return a topic_document dataframe
    '''
    #read in jobs file and get descriptions
    df = pd.read_csv('jobs.csv')
    
    #df = df[df.keyword!='marketing']
    jobs_df = pd.DataFrame(zip(df['Job Description'], df['keyword']), columns = ['Description', 'Job'])

    array, doc, topic_model, vec, topic_list  = return_topics(jobs_df['Description'],20, 10, TruncatedSVD, TfidfVectorizer)

    topic_df = pd.DataFrame(doc)
    topic_df.columns = ['Topic ' + str(i+1) for i in range(len(topic_df.columns)) ]

    topic_df['job'] = jobs_df.Job
    #Topic_DF.to_csv('topic_df.csv')
    return topic_df, topic_model, vec, topic_list